# PairSpec-dLLM 評価ノートブック

Google Drive 上に配置済みの Fast-dLLM/PairSpec-dLLM を直接参照し、
PairSpec（先行ドラフト併走）あり/なしの生成性能を **GSM8K (openai/gsm8k)** と **HumanEval (openai/openai_humaneval)** で比較します。

- 速度系指標: latency / tokens per second / forward passes (NFE)
- 精度系指標: GSM8K Accuracy, HumanEval Pass@1
- モデルは Drive 上にダウンロード済みのものを直接使用（ローカル複製なし）
- PairSpec 有効時は draft モデルを別 GPU/同 GPU 上で並列起動します

## 0. 手順概要
1. Google Drive をマウントし、既存の `PairSpec-dLLM` ルートへ `os.chdir`。
2. 依存関係をインストールし、（必要なら）Hugging Face でログイン。
3. 評価用の共通設定（モデルパス・ブロック長・サンプル数など）を記入。
4. ノートブック内の関数でベースライン / PairSpec の両方を実行。
5. 指標サマリとサンプルごとの詳細ログを確認。

## 1. Google Drive をマウント（※既にマウント済みならスキップ可）

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. 作業ディレクトリと環境変数のセット
- `DRIVE_PROJECT_DIR` を Drive 上の `PairSpec-dLLM` ルートに変更してください。
- 既存ファイルをそのまま参照するため、ローカルへの rsync/コピーは行いません。

In [ ]:
import os, sys

DRIVE_PROJECT_DIR = '/content/drive/MyDrive/PairSpec-dLLM'  # ★必要に応じて変更

if not os.path.isdir(DRIVE_PROJECT_DIR):
    raise FileNotFoundError(f'{DRIVE_PROJECT_DIR} が見つかりません。パスを確認してください。')

os.chdir(DRIVE_PROJECT_DIR)
if DRIVE_PROJECT_DIR not in sys.path:
    sys.path.insert(0, DRIVE_PROJECT_DIR)

os.environ['HF_ALLOW_CODE_EVAL'] = '1'
os.environ['HF_DATASETS_TRUST_REMOTE_CODE'] = '1'
print('Working directory:', os.getcwd())

## 3. 依存パッケージのインストール
- プロジェクト付属の `requirements.txt` に加え、評価で用いる `datasets` / `evaluate` / `accelerate` を最新化します。

In [ ]:
!pip install -U pip
!pip install -r requirements.txt
!pip install -U datasets evaluate accelerate huggingface_hub

## 4. （必要に応じて）Hugging Face にログイン
- private/gated モデルを Drive に保存済みであればスキップ可。
- Hub から直接取得する際は以下を実行し、トークンを入力してください。

In [ ]:
from huggingface_hub import login
login()  # ★Hub からダウンロードする場合のみ実行

## 5. 評価設定を記入
- `MAIN_MODEL_PATH` / `DRAFT_MODEL_PATH` を Drive 内のダウンロード済みモデルに変更してください（HF Hub ID でも可）。
- サンプル数を絞りたい場合は `GSM8K_MAX_SAMPLES` / `HUMAN_MAX_SAMPLES` を小さめに設定します（`None` で全件）。
- PairSpec の受理ポリシーやドラフト深度もここで調整します。

In [ ]:
from types import SimpleNamespace

# === モデル/デバイス設定 ===
MAIN_MODEL_PATH = 'GSAI-ML/LLaDA-8B-Instruct'  # 例: ローカル格納済みモデル or HF Repo
DRAFT_MODEL_PATH = 'GSAI-ML/LLaDA-1.5'   # 例: ドラフト用ローカルモデル or HF Repo
VERIFY_DEVICE = 'cuda:0'
DRAFT_DEVICE = 'cuda:0'  # Colab では単一 GPU のため同一デバイスを想定

# === 生成パラメータ ===
GEN_LENGTH = 128
BLOCK_SIZE = 32
BASELINE_STEPS = 128        # 通常 Fast-dLLM のステップ数
PARALLEL_THRESHOLD = 0.9    # 信頼度しきい値（必要に応じて調整）

# === PairSpec 固有 ===
PAIRSPEC_ACCEPT_POLICY = 'lossless'  # 'lossless' or 'thresholded'
PAIRSPEC_ACCEPT_THRESHOLD = 2.0
PAIRSPEC_DRAFT_DEPTH = 2
PAIRSPEC_DRAFT_STEPS = None  # None -> 自動で steps/num_blocks

# === 評価データ設定 ===
GSM8K_MAX_SAMPLES = 32        # None で全テストセット (1319)
HUMAN_MAX_SAMPLES = 32        # None で全 164 件
HUMAN_EVAL_TIMEOUT = 15       # コード実行のタイムアウト（秒）
HUMAN_EVAL_MAX_WORKERS = 4    # 同時テスト実行数

BASELINE_ARGS = SimpleNamespace(
    gen_length=GEN_LENGTH,
    steps=BASELINE_STEPS,
    block_size=BLOCK_SIZE,
    use_cache=True,
    if_cache_position=True,
    threshold=PARALLEL_THRESHOLD,
)

PAIRSPEC_ARGS = SimpleNamespace(
    gen_length=GEN_LENGTH,
    steps=BASELINE_STEPS,
    block_size=BLOCK_SIZE,
    use_cache=True,
    if_cache_position=True,
    threshold=PARALLEL_THRESHOLD,
    draft_model=DRAFT_MODEL_PATH,
    draft_device=DRAFT_DEVICE,
    draft_depth=PAIRSPEC_DRAFT_DEPTH,
    draft_steps=PAIRSPEC_DRAFT_STEPS,
    accept_policy=PAIRSPEC_ACCEPT_POLICY,
    accept_threshold=PAIRSPEC_ACCEPT_THRESHOLD,
)

print('設定完了: main model =', MAIN_MODEL_PATH)

## 5.1 モデルパスの解決（ローカル優先）
- 指定ディレクトリが存在しない場合は Hugging Face から Drive に自動ダウンロードします。

In [ ]:
from huggingface_hub import snapshot_download

LOCAL_MODEL_CACHE_ROOT = '/content/drive/MyDrive/hf_models'

def ensure_local_model(identifier: str, cache_root: str = LOCAL_MODEL_CACHE_ROOT):
    if identifier is None or identifier == '':
        return None
    if os.path.isdir(identifier):
        print(f"ローカルパスを使用: {identifier}")
        return identifier
    safe_name = identifier.replace('/', '__')
    target_dir = os.path.join(cache_root, safe_name)
    if os.path.isdir(target_dir):
        print(f"既存のキャッシュを使用: {target_dir}")
        return target_dir
    os.makedirs(target_dir, exist_ok=True)
    print(f"Hugging Face から {identifier} をダウンロード -> {target_dir}")
    snapshot_download(repo_id=identifier, local_dir=target_dir, local_dir_use_symlinks=False)
    return target_dir

MAIN_MODEL_PATH_RESOLVED = ensure_local_model(MAIN_MODEL_PATH)
DRAFT_MODEL_PATH_RESOLVED = ensure_local_model(DRAFT_MODEL_PATH)
PAIRSPEC_ARGS.draft_model = DRAFT_MODEL_PATH_RESOLVED
print('Main model path:', MAIN_MODEL_PATH_RESOLVED)
print('Draft model path:', DRAFT_MODEL_PATH_RESOLVED)

## 6. モデルとトークナイザの読み込み
- Drive 上にあるモデルフォルダを `from_pretrained` で直接指定します。
- bfloat16 / eval モードで読み込みます。

In [ ]:
import torch
from transformers import AutoTokenizer
from llada.model.modeling_llada import LLaDAModelLM

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if str(device) != VERIFY_DEVICE:
    print(f"Warning: 要求された VERIFY_DEVICE={VERIFY_DEVICE} と実際の device={device} が異なります。")

MODEL_LOAD_PATH = MAIN_MODEL_PATH_RESOLVED or MAIN_MODEL_PATH
if MODEL_LOAD_PATH is None:
    raise ValueError("MAIN_MODEL_PATH を設定してください。")

tokenizer = AutoTokenizer.from_pretrained(MODEL_LOAD_PATH, trust_remote_code=True)
model = LLaDAModelLM.from_pretrained(
    MODEL_LOAD_PATH,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
).to(device).eval()

print('モデル / トークナイザ読み込み完了')

## 7. ヘルパー関数の定義
- プロンプト生成、最終数値抽出、PairSpec 生成クラス、評価ループなどを実装します。
- PairSpec 生成は `specdraft` モジュールを直接利用し、ドラフトワーカを使い回せるようクラス化しています。

In [ ]:
import math
import re
import time
import json
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
from llada.chat import _select_generator_name, _select_generator_fn
from specdraft.dispatcher import DraftRequest, start_draft_worker, shutdown_draft_worker
from specdraft.acceptor import compute_prefix_hash, verify_and_commit
from specdraft.kv_manager import KVManager





def strip_code_fence(text: str) -> str:
    stripped = text.strip()
    if stripped.startswith("```"):
        lines = stripped.splitlines()
        if lines and lines[0].startswith("```"):
            lines = lines[1:]
        while lines and lines[-1].strip() == "```":
            lines = lines[:-1]
        stripped = "\n".join(lines)
    return stripped.strip()



MASK_TOKEN_ID = tokenizer.mask_token_id or 126336

def apply_chat_template(user_text: str) -> str:
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_text.strip()},
    ]
    return tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

def build_gsm8k_prompt(question: str) -> str:
    instruction = (
        "Solve the math word problem step by step and output the final numeric answer in the format '#### value'.\n\n"
        f"Problem: {question.strip()}\n\nAnswer:"
    )
    return apply_chat_template(instruction)

def build_humaneval_prompt(sample: dict) -> str:
    instruction = (
        "Complete the Python function specification below. Return only executable Python code.\n\n"
        f"{sample['prompt'].strip()}"
    )
    return apply_chat_template(instruction)

def extract_gsm8k_answer(text: str):
    matches = re.findall(r"####\s*([-+]?\d+(?:\.\d+)?)", text)
    if matches:
        try:
            return float(matches[-1])
        except ValueError:
            return None
    return None

def count_new_tokens(tokens: torch.Tensor) -> int:
    if tokens is None or tokens.numel() == 0:
        return 0
    return tokens.shape[1]

def run_baseline_generation(prompt_ids: torch.Tensor, args) -> tuple:
    generator_name = _select_generator_name(args)
    generator_fn = _select_generator_fn(generator_name)
    out, nfe = generator_fn(
        model,
        prompt_ids,
        steps=args.steps,
        gen_length=args.gen_length,
        block_length=args.block_size,
        temperature=0.0,
        remasking='low_confidence',
        threshold=args.threshold,
    )
    new_tokens = out[:, prompt_ids.shape[1]:]
    stats = {
        'accepted_blocks': 0,
        'attempted_blocks': args.gen_length // args.block_size,
        'draft_nfe': 0,
        'verify_nfe': 0,
        'fallback_nfe': nfe,
        'total_nfe': nfe,
    }
    return new_tokens, nfe, stats

def run_block_generation(current_prompt: torch.Tensor, args, steps_per_block: int, generator_name: str):
    generator_fn = _select_generator_fn(generator_name)
    out, nfe = generator_fn(
        model,
        current_prompt,
        steps=steps_per_block,
        gen_length=args.block_size,
        block_length=args.block_size,
        temperature=0.0,
        remasking='low_confidence',
        threshold=args.threshold,
    )
    block_tensor = out[:, -args.block_size:]
    return block_tensor, int(nfe)

class PairSpecSession:
    def __init__(self, args, tokenizer):
        self.args = args
        self.tokenizer = tokenizer
        self.generator_name = _select_generator_name(args)
        self.worker, self.request_queue, self.spec_queue = start_draft_worker(
            backend='llada',
            model_path=args.draft_model,
            device=args.draft_device,
            generator=self.generator_name,
            max_depth=args.draft_depth,
            dtype=torch.bfloat16,
        )

    def close(self):
        shutdown_draft_worker(self.worker, self.request_queue)
        if self.spec_queue:
            self.spec_queue.close()

    def generate(self, prompt_ids: torch.Tensor):
        args = self.args
        block_size = args.block_size
        gen_length = args.gen_length
        if gen_length % block_size != 0:
            raise ValueError('gen_length は block_size で割り切れる必要があります')
        total_blocks = gen_length // block_size
        steps_per_block = max(1, args.steps // max(total_blocks, 1))
        kv_mgr = KVManager(model)
        prefix_hash = compute_prefix_hash(prompt_ids[0].tolist())
        stats = {
            'accepted_blocks': 0,
            'attempted_blocks': total_blocks,
            'draft_nfe': 0,
            'verify_nfe': 0,
            'fallback_nfe': 0,
        }
        pending_blocks = set()

        def enqueue(block_id: int, current_prompt: torch.Tensor, current_hash: str):
            if block_id >= total_blocks or block_id in pending_blocks:
                return
            req = DraftRequest(
                block_id=block_id,
                prefix_tokens=current_prompt[0].tolist(),
                prefix_hash=current_hash,
                block_size=block_size,
                steps_per_block=max(1, args.draft_steps or steps_per_block),
                temperature=0.0,
                remasking='low_confidence',
                threshold=args.threshold,
                generator=self.generator_name,
            )
            self.request_queue.put(req)
            pending_blocks.add(block_id)

        enqueue(0, prompt_ids, prefix_hash)
        current_prompt = prompt_ids.clone()
        generated_segments = []

        for block_id in range(total_blocks):
            draft = self.spec_queue.try_get(block_id)
            if draft is not None:
                pending_blocks.discard(block_id)
                stats['draft_nfe'] += draft.nfe
                if draft.prefix_hash != prefix_hash:
                    draft = None

            accepted_tensor = None
            if draft is not None:
                verification = verify_and_commit(
                    draft,
                    model,
                    current_prompt,
                    mask_token_id=MASK_TOKEN_ID,
                    policy=args.accept_policy,
                    threshold=args.accept_threshold,
                )
                stats['verify_nfe'] += verification.nfe
                if verification.accepted:
                    stats['accepted_blocks'] += 1
                    accepted_tensor = torch.tensor(
                        verification.accepted_tokens,
                        dtype=current_prompt.dtype,
                        device=current_prompt.device,
                    ).unsqueeze(0)

            if accepted_tensor is None:
                block_tensor, nfe_block = run_block_generation(
                    current_prompt,
                    args,
                    steps_per_block,
                    self.generator_name,
                )
                stats['fallback_nfe'] += nfe_block
            else:
                block_tensor = accepted_tensor

            current_prompt = torch.cat([current_prompt, block_tensor], dim=1)
            prefix_hash = compute_prefix_hash(current_prompt[0].tolist())
            kv_mgr.recompute_on_commit(prefix_hash)
            generated_segments.append(block_tensor)
            enqueue(block_id + 1, current_prompt, prefix_hash)

        full_generation = torch.cat(generated_segments, dim=1) if generated_segments else torch.empty((1, 0), dtype=current_prompt.dtype, device=current_prompt.device)
        stats['total_nfe'] = stats['verify_nfe'] + stats['fallback_nfe']
        return full_generation, stats['total_nfe'], stats

def generate_text(prompt_text: str, mode: str, pair_session: PairSpecSession = None):
    encoded = tokenizer(prompt_text, return_tensors='pt').input_ids.to(device)
    start = time.perf_counter()
    if mode == 'pairspec':
        if pair_session is None:
            raise ValueError('PairSpec モードには PairSpecSession が必要です')
        new_tokens, nfe, stats = pair_session.generate(encoded)
    else:
        new_tokens, nfe, stats = run_baseline_generation(encoded, BASELINE_ARGS)
    elapsed = time.perf_counter() - start
    decoded = tokenizer.batch_decode(new_tokens, skip_special_tokens=True)[0].strip()
    token_count = count_new_tokens(new_tokens)
    tps = token_count / elapsed if elapsed > 0 else float('inf')
    metrics = {
        'latency': elapsed,
        'tokens': token_count,
        'tps': tps,
        'nfe': nfe,
        **stats,
    }
    return decoded, metrics

print('ヘルパー関数を定義しました。')

## 8. 評価ルーチン
- GSM8K: 最終数値一致で正解判定。
- HumanEval: Hugging Face `code_eval` を用いて pass@1 を算出。
- 速度指標はサンプルごとのログから集計します。

In [ ]:
import evaluate

def evaluate_gsm8k(mode: str, max_samples: int = None, pair_session: PairSpecSession = None):
    dataset = load_dataset('openai/gsm8k', 'main', split='test')
    if max_samples is not None:
        dataset = dataset.select(range(min(max_samples, len(dataset))))
    records = []
    for idx, sample in enumerate(tqdm(dataset, desc=f'GSM8K [{mode}]')):
        prompt = build_gsm8k_prompt(sample['question'])
        completion, metrics = generate_text(prompt, mode, pair_session)
        pred = extract_gsm8k_answer(completion)
        gold = extract_gsm8k_answer(sample['answer'])
        correct = int(pred is not None and gold is not None and math.isclose(pred, gold))
        records.append({
            'dataset': 'gsm8k',
            'mode': mode,
            'index': idx,
            'question': sample['question'],
            'gold_answer': gold,
            'prediction_text': completion,
            'prediction_value': pred,
            'correct': correct,
            **metrics,
        })
    df = pd.DataFrame(records)
    summary = {
        'dataset': 'gsm8k',
        'mode': mode,
        'samples': len(df),
        'accuracy': df['correct'].mean() if len(df) else float('nan'),
        'avg_latency': df['latency'].mean() if len(df) else float('nan'),
        'avg_tps': df['tps'].mean() if len(df) else float('nan'),
        'avg_nfe': df['nfe'].mean() if len(df) else float('nan'),
        'accepted_block_ratio': (df['accepted_blocks'] / df['attempted_blocks']).mean() if 'accepted_blocks' in df else 0.0,
    }
    return df, summary

def evaluate_humaneval(mode: str, max_samples: int = None, pair_session: PairSpecSession = None):
    dataset = load_dataset('openai/openai_humaneval', split='test')
    if max_samples is not None:
        dataset = dataset.select(range(min(max_samples, len(dataset))))
    records = []
    predictions = []
    references = []
    for idx, sample in enumerate(tqdm(dataset, desc=f'HumanEval [{mode}]')):
        prompt = build_humaneval_prompt(sample)
        completion, metrics = generate_text(prompt, mode, pair_session)
        clean_completion = strip_code_fence(completion)
        predictions.append(json.dumps({
            'task_id': sample['task_id'],
            'completion': clean_completion,
        }))
        references.append(json.dumps({
            'task_id': sample['task_id'],
            'prompt': sample['prompt'],
            'canonical_solution': sample.get('canonical_solution', ''),
            'test': sample['test'],
            'entry_point': sample['entry_point'],
        }))
        records.append({
            'dataset': 'humaneval',
            'mode': mode,
            'task_id': sample['task_id'],
            'prompt': sample['prompt'],
            'completion': clean_completion,
            **metrics,
        })

    code_eval = evaluate.load('code_eval')
    eval_result = code_eval.compute(
        references=references,
        predictions=predictions,
        k=[1],
        timeout=HUMAN_EVAL_TIMEOUT,
    )
    pass_dict = eval_result.get('pass@k') or {}
    pass_at_1 = pass_dict.get(1, eval_result.get('pass@1', float('nan')))

    df = pd.DataFrame(records)
    summary = {
        'dataset': 'humaneval',
        'mode': mode,
        'samples': len(df),
        'pass@1': pass_at_1,
        'avg_latency': df['latency'].mean() if len(df) else float('nan'),
        'avg_tps': df['tps'].mean() if len(df) else float('nan'),
        'avg_nfe': df['nfe'].mean() if len(df) else float('nan'),
        'accepted_block_ratio': (df['accepted_blocks'] / df['attempted_blocks']).mean() if 'accepted_blocks' in df else 0.0,
    }
    return df, summary, eval_result

print('Evaluation routines ready.')



## 9. ベースライン & PairSpec 評価を実行
- それぞれのデータセットについて、PairSpec セッションを初期化してから計測します。
- 実行には時間がかかるので、`MAX_SAMPLES` を調整しながら進めてください。

In [ ]:
all_dfs = []
summaries = []

# --- ベースライン ---
gsm8k_baseline_df, gsm8k_baseline_summary = evaluate_gsm8k('baseline', GSM8K_MAX_SAMPLES)
all_dfs.append(gsm8k_baseline_df)
summaries.append(gsm8k_baseline_summary)

humaneval_baseline_df, humaneval_baseline_summary, humaneval_baseline_eval = evaluate_humaneval('baseline', HUMAN_MAX_SAMPLES)
all_dfs.append(humaneval_baseline_df)
summaries.append(humaneval_baseline_summary)

# --- PairSpec ---
pairspec_session = PairSpecSession(PAIRSPEC_ARGS, tokenizer)
try:
    gsm8k_pairspec_df, gsm8k_pairspec_summary = evaluate_gsm8k('pairspec', GSM8K_MAX_SAMPLES, pairspec_session)
    all_dfs.append(gsm8k_pairspec_df)
    summaries.append(gsm8k_pairspec_summary)

    humaneval_pairspec_df, humaneval_pairspec_summary, humaneval_pairspec_eval = evaluate_humaneval('pairspec', HUMAN_MAX_SAMPLES, pairspec_session)
    all_dfs.append(humaneval_pairspec_df)
    summaries.append(humaneval_pairspec_summary)
finally:
    pairspec_session.close()

full_results_df = pd.concat(all_dfs, ignore_index=True)
summary_df = pd.DataFrame(summaries)
print('評価完了')

## 10. 指標サマリと詳細ログ
- `summary_df` でデータセット毎の主要指標を確認できます。
- `full_results_df` をフィルタすればサンプルごとのログ（正誤・速度・PairSpec 受理率など）を参照できます。
- 必要に応じて CSV で Drive に保存してください。

In [ ]:
summary_df

In [ ]:
# 例: 先頭5件だけ表示
full_results_df.head()

In [ ]:
# 任意: CSV として Drive に保存
# summary_df.to_csv('/content/drive/MyDrive/pairspec_eval_summary.csv', index=False)
# full_results_df.to_csv('/content/drive/MyDrive/pairspec_eval_details.csv', index=False)
print('必要に応じて CSV 保存コマンドを有効化してください。')

## 11. 追加メモ
- HumanEval の `code_eval` は Python コードを実行するため、信頼できる環境でのみ実行してください。
- PairSpec のドラフト/検証を単一 GPU で同時に走らせるとメモリ使用量が増えるため、必要に応じて `GEN_LENGTH` や `BLOCK_SIZE`、`draft_depth` を調整してください。
- `PAIRSPEC_ARGS` の `accept_policy='thresholded'` に切り替えると、ドラフト受理を信頼度しきい値ベースに変更できます。